In [8]:
### mount on google drive
from google.colab import drive
drive.mount('/content/drive/')
# go to your code directory
import os
os.chdir("/content/drive/My Drive/Earth-surface-water-mapping")
# !ls
# !nvidia-smi


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [9]:
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf
import numpy as np
import time
import random
from utils.utils import acc_patch, imgShow
from dataloader.dataloader import get_scene, get_patch, image_aug
from models.models import MobileNetV2, DeepLabV3Plus_improve


In [3]:
## super parameters
PATCH_SIZE = 512
EPOCHS = 500
lr = 0.005
BATCH_SIZE = 4
BUFFER_SIZE = 200

In [4]:
### get training data
## previous, 8s/epoch
# # training dataset (!!your dataset(.tif file) directory)
# folder_TrainScenes = '/content/drive/My Drive/Colab/WaterMapping/TrainingData/TrainingScene/'
# folder_TrainTruths = '/content/drive/My Drive/Colab/WaterMapping/TrainingData/TrainingTruth/'
# Scenes, Truths = get_scene(folder_TrainScenes, folder_TrainTruths, PATCH_SIZE)
# traData = get_patch(Scenes, Truths, PATCH_SIZE, BATCH_SIZE, BUFFER_SIZE)


## Load and parse the tfrecord data

In [15]:
### training dataset (!!your dataset(.tfrecords file) directory)
record_file = '/content/drive/My Drive/Colab/WaterMapping/TrainingData/traData.tfrecords'

traData = tf.data.TFRecordDataset(record_file)
feaBand = ['blueBand','greenBand','redBand','nirBand','swir1Band','swir2Band']
truBand = ['truth']
mergeBand = feaBand+truBand

featuresDict = {
    'bandShape': tf.io.FixedLenFeature([2,],dtype=tf.int64),
    'blueBand': tf.io.VarLenFeature(dtype=tf.float32),
    'greenBand': tf.io.VarLenFeature(dtype=tf.float32),
    'redBand': tf.io.VarLenFeature(dtype=tf.float32),
    'nirBand': tf.io.VarLenFeature(dtype=tf.float32),
    'swir1Band': tf.io.VarLenFeature(dtype=tf.float32),
    'swir2Band': tf.io.VarLenFeature(dtype=tf.float32),
    'truth': tf.io.VarLenFeature(dtype=tf.float32),
}

def parse_image(example_proto):
    # Parse the input tf.train.Example proto using the dictionary above.
    return tf.io.parse_single_example(example_proto, featuresDict)

def parse_shape(example_parsed):
    for fea in mergeBand:
        example_parsed[fea] = tf.sparse.to_dense(example_parsed[fea])
        example_parsed[fea] = tf.reshape(example_parsed[fea], example_parsed['bandShape'])
    return example_parsed

def toPatchPair(inputs):
    inputsList = [inputs.get(key) for key in mergeBand]
    stacked = tf.stack(inputsList, axis=2)
    cropped_stacked = tf.image.random_crop(
                    stacked, size=[512, 512, len(mergeBand)])
    return cropped_stacked[:,:,:len(feaBand)], cropped_stacked[:,:,len(feaBand):]

traData = traData.map(parse_image).map(parse_shape)\
                                    .map(toPatchPair).map(image_aug)
traData = traData.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
traData

<BatchDataset shapes: ((None, 512, 512, 6), (None, 512, 512, 1)), types: (tf.float32, tf.float32)>

In [16]:
## model configuration
model = DeepLabV3Plus_improve(input_shape=(PATCH_SIZE, PATCH_SIZE, 6), base_model=MobileNetV2,  
                      d_feature=91, m_feature=24, l_feature=11, nclasses=2)
model_loss = tf.keras.losses.BinaryCrossentropy()

*** Building DeepLabv3Plus Network ***
*** Output_Shape => (None, 512, 512, 1) ***


In [18]:
def train_step(TraSet, lr):
    TraAcc_OA_epoch = []
    TraAcc_MIoU_epoch = []
    Train_loss_epoch = []
    model_optimizer = tf.keras.optimizers.Adam(lr)
    for Train_Patch, Train_Truth in TraSet:
        with tf.GradientTape() as tape:
            outp_TraPatch = model(Train_Patch, training=True)  
            outp_loss = model_loss(Train_Truth, outp_TraPatch)
            model_gradients = tape.gradient(outp_loss, model.trainable_variables)
            model_optimizer.apply_gradients(zip(model_gradients, model.trainable_variables))
            Train_loss_epoch.append(outp_loss)
            TraAcc_OA, TraAcc_MIoU = acc_patch(Train_Truth, outp_TraPatch)
            TraAcc_OA_epoch.append(TraAcc_OA)
            TraAcc_MIoU_epoch.append(TraAcc_MIoU)
    Train_loss = np.mean(np.array(Train_loss_epoch)) 
    TraAcc_OA = np.mean(np.array(TraAcc_OA_epoch))
    TraAcc_MIoU = np.mean(np.array(TraAcc_MIoU_epoch))
    return Train_loss, TraAcc_OA, TraAcc_MIoU

def train_epoch(TraSet,EPOCHS):
    Train_loss_plot = []
    TraAcc_OA_plot = []
    TraAcc_MIoU_plot = []
    for epoch in range(EPOCHS):
        start = time.time() 
        lr_new = lr*((1-epoch/EPOCHS)**0.9)
        Train_loss_step, TraAcc_OA_step, TraAcc_MIoU_step = train_step(TraSet, lr_new)
        print('Epoch: {}, Train loss: {}, Train OA: {}, Train MIoU: {}, Time taken: {} sec'.format(
            epoch + 1, Train_loss_step, TraAcc_OA_step, TraAcc_MIoU_step, time.time() - start))
        Train_loss_plot.append(Train_loss_step)
        TraAcc_OA_plot.append(TraAcc_OA_step)
        TraAcc_MIoU_plot.append(TraAcc_MIoU_step)
    return Train_loss_plot, TraAcc_OA_plot, TraAcc_MIoU_plot


In [ ]:
####  model training and save the trained model (!!change to the user's directory)

train_epoch(traData,EPOCHS)

# model.save('/content/drive/My Drive/Earth-surface-water-mapping/pretrained_model/pretrained_model_v1')